In [1]:
import hkvwaporpy as hkv
import numpy as np
import pandas as pd
import glob
import os
from tqdm import tqdm_notebook as tqdm

In [2]:
csv_in = r'D:\jupyter notebooks\3348.10 WAP Awash - Ethiopie\CSV-files\*.csv'
csv_dir = glob.glob(csv_in)

In [3]:
for csv in tqdm(csv_dir):
    print(csv)
    path_basename = os.path.basename(csv)
    path_dirname = os.path.dirname(csv)
    csv_out = '{0}\select_stats\select_{1}'.format(path_dirname, path_basename)    
    df = pd.read_csv(csv)
    # parse date column as datetime
    df.loc[:,'date'] = pd.to_datetime(df['date'])
    # compute year and day of year column
    df.loc[:,'year'] = df['date'].dt.year
    df.loc[:,'doy'] = df['date'].dt.dayofyear   
    
    # iterate over unique weredas and day of years to select subset
    unique_wereda = df['wereda'].unique()
    unique_doy = df['doy'].unique()

    if 'stats_L1_RET' not in path_basename:
        # only get doy that are minimal 8 days apart
        diff_day = np.ediff1d(unique_doy)
        diff_day = np.insert(diff_day, 0, 10)
        unique_doy = unique_doy[diff_day>=8]

    # extract mean of each variable for each wereda and doy and store in list
    list_comb = []
    for wereda in tqdm(unique_wereda):
        for doy in tqdm(unique_doy):
            
            if 'stats_L1_RET' not in path_basename:
                # search surround
                df_sel = df.loc[(df['doy'].isin([doy-1,doy, doy+1])) & (df['wereda'] == wereda)]
            else:
                df_sel = df.loc[(df['doy'] == doy) & (df['wereda'] == wereda)]
            s_sel_mean = df_sel.mean(axis=0)        
            s_sel_mean.loc['wereda'] = df_sel.wereda.unique()[0]
            s_sel_mean.loc['doy'] = df_sel.doy.min()
            list_comb.append(s_sel_mean.tolist())

    # convert list to dataframe
    df_comb = pd.DataFrame(data = list_comb, columns = s_sel_mean.index)      
    df_comb.drop(['Unnamed: 0', 'year'], axis=1, inplace=True)
    df_comb.loc[:,'parameter'] = path_basename[6:-4]
    df_comb.to_csv(csv_out, index=False)

D:\jupyter notebooks\3348.10 WAP Awash - Ethiopie\CSV-files\stats_L1_RET.csv
D:\jupyter notebooks\3348.10 WAP Awash - Ethiopie\CSV-files\stats_L2_AET_D.csv
D:\jupyter notebooks\3348.10 WAP Awash - Ethiopie\CSV-files\stats_L2_AGBP_S.csv


Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\python35\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "c:\python35\lib\site-packages\tqdm\_tqdm.py", line 147, in run
    if instance.miniters > 1 and \
AttributeError: 'tqdm_notebook' object has no attribute 'miniters'



D:\jupyter notebooks\3348.10 WAP Awash - Ethiopie\CSV-files\stats_L2_NPP.csv
D:\jupyter notebooks\3348.10 WAP Awash - Ethiopie\CSV-files\stats_L2_PHE.csv
D:\jupyter notebooks\3348.10 WAP Awash - Ethiopie\CSV-files\stats_L2_QUAL_NDVI.csv
D:\jupyter notebooks\3348.10 WAP Awash - Ethiopie\CSV-files\stats_L2_TFRAC.csv

